# Homies App UI

**In this notebook, we build a simple yet elegant user interface in order to make the user experience more fluid. Run the entire notebook in order to load the data then to display the interactive interface**

In [1]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='hdfs://iccluster044.iccluster.epfl.ch', port=8020, user='ebouille')
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output, Layout
import ipywidgets as widgets
import pandas as pd
import numpy as np
import datetime
import pickle

**We create two simple functions in order to read the files previously prepared in final_notebook.ipynb**

In [2]:
def read_file(name):
    with hdfs.open(name, "rb") as f:
        x = pickle.load(f)
    print('File loaded')
    return x
def read_df(name):
    with hdfs.open(name, "rb") as f:
        x = pd.read_pickle(f)
    print('File loaded')
    return x

**We load the precomputed data.**

In [3]:
dict_edges_bus = read_file('/user/{}/dict_edges_simple.pkl'.format('kooli'))
dict_edges_waiting = read_file('/user/{}/dict_edges_corr.pkl'.format('kooli'))
vertices = read_df('/user/{}/pd_vertices.pkl'.format('kooli'))
df_index = read_df('/user/{}/df_index.pkl'.format('kooli'))
walking_neighbors = read_file('/user/{}/dict_walking_neighbors.pkl'.format('kooli') )
dict_walk_time = read_file('/user/{}/dict_walk_time.pkl'.format('kooli'))
dict_stop_list_hours =  read_file('/user/{}/dict_stop_list_hours.pkl'.format('kooli'))
df_medians = read_df('/user/{}/pd_delay_dist1yearmeanperh.pkl'.format('kooli'))
list_station_zurich = read_file('/user/{}/list_stations_zurich.pkl'.format('kooli'))

File loaded
File loaded
File loaded
File loaded
File loaded
File loaded
File loaded
File loaded
File loaded


**Oops! We need to modify just a little bit the vertices dataframe but don't worry it is really fast.**

In [4]:
vertices['station'] =  vertices.id.map(lambda x :  x.split('_')[0])
vertices['time'] = vertices.id.map(lambda x :  x.split('_')[1])
vertices['time_formatted'] = vertices.time.map(lambda x : datetime.datetime.strptime('2019-05-13'+' '+x, "%Y-%m-%d %H:%M:%S"))

**Now let's load functions bfs_homies, get_safe_paths and get_final_output that have been used in final_notebook.ipynb.**
**For practicity they have been rewritten to work on Python in utility.py.**

In [5]:
from utility import *

**The following cell is used to create the user interface with Ipywidgets.**

As you can notice the arrival time is bounded between 8:00 and 16:59 and the stops you want find a journey between are expect in ID format.
The maximum number of iteration through BFS as been set to 10 in order to have avoid any long calculation but feel free to change it. If it is too low don't worry the algorithm will warn you. 
The user can specify a minimum number of paths that BFS returns before taking the risk into account. Its default value is 8 but the algorithm might warn you to increase it depending on how much it found.
Last but not least, the slider allows you to select the confidence interval you want to filter your journeys on. Notice that in order to avoid the unuseful run of the BFS algorithm, that if you only change the value of the confidence interval the callback search will only call get_safe_paths and get_final_output.
Give it a try! Fill the boxes and observe this complex algorithm computes the best journey and displaying them from the latest departure time!

In [6]:
# Wigdets for the selection of the maximum_arrival_time
start_hour = widgets.BoundedIntText(value = 12, min = 8, max = 16, step = 1, description = 'Hour:Minute', disabled = False,\
                                    layout = Layout(width = 'auto', height = '30px'))
start_minute = widgets.BoundedIntText(value = 30,min = 0,max = 59, step = 1, disabled = False, layout = Layout(width='auto', height='30px'))

# Wigdets for the selection of the BFS parameters
max_iter = widgets.BoundedIntText(value = 15, min = 5, max = 30, step = 1, description = 'Number maximum of iteration for BFS',\
                                  disabled = False, layout = Layout(width = 'auto', height = '30px'),style = {'description_width': 'initial'})
minimum_paths = widgets.BoundedIntText(value = 15, min = 5,max = 50, step = 1, description = 'Minimum paths before taking the risk into account', disabled = False, \
                                       layout = Layout(width = 'auto', height = '30px'),style = {'description_width': 'initial'})

# Wigdets for the selection of the stations ID
start_stop = widgets.Combobox(value = '8503000', placeholder = 'Select a station', options = list_station_zurich, description = 'From:',\
                              ensure_option = True, disabled = False, layout = Layout(width='19%', height='30px'))
end_stop = widgets.Combobox(value = '8591049', placeholder = 'Select a station', options = list_station_zurich, description = 'To:',\
                            ensure_option = True, disabled = False, layout = Layout(width = '19%', height = '30px'))

# Wigdets for the selection of the risk you are willing to take
conf = widgets.FloatSlider(value=70,min=0,max=100,step=1,description='',disabled=False,\
                             continuous_update=False,orientation='horizontal',readout=True,readout_format='.1f',)

# Button widget that call the callback search.
valid = widgets.Button(description='Search', disabled=False, button_style='primary',tooltip='Go ahead and click',icon='search')

# Specify where the output should appear
out = widgets.Output()

def search(b):
    out.clear_output()
    with out:
        
        # If anything except conf changed rerun the BFS algorithm
        if((search.start_hour != start_hour.value) or (search.start_minute != start_minute.value) \
           or (search.start_stop != start_stop.value) or (search.end_stop != end_stop.value) \
           or (search.minimum_paths != minimum_paths.value) or (search.max_iter != max_iter.value)):
            
            # We save these inputs in the functions in order to check on the next call
            search.start_hour, search.start_minute, search.start_stop ,search.end_stop, search.minimum_paths, search.max_iter = start_hour.value, start_minute.value, start_stop.value ,end_stop.value, minimum_paths.value, max_iter.value
            
            # We make a string with Hour and Minute
            search.max_time_arrival = str(search.start_hour) + ":"+ str(search.start_minute)+ ":00"
            
            # We find all possible paths with BFS algorithm
            search.true_paths, search.transports = bfs_homies(search.start_stop, search.end_stop, search.max_time_arrival, dict_edges_bus,
                                                                dict_edges_waiting, vertices, df_index, walking_neighbors,
                                                                   dict_walk_time, dict_stop_list_hours, search.max_iter, search.minimum_paths)

        # We find the safe paths within the confidence interval
        path_safe , safe_probas = get_safe_paths(search.true_paths, search.transports,search.max_time_arrival,conf.value,df_medians, dict_walk_time)
        
        print("")
        
        # We find the message to display
        final_output , output_messages = get_final_output(path_safe,safe_probas)
        for msg in output_messages:
            print(msg)
            print("")
        
# We set the function parameters to 0 to be sure we run BFS for the first call      
search.start_hour, search.start_minute, search.start_stop ,search.end_stop, search.minimum_paths, search.max_iter = 0,0,0,0,0,0
valid.on_click(search)
hbox0 = widgets.HBox([max_iter, minimum_paths])
hbox1 = widgets.HBox([widgets.Label(value="Arrives at:", layout=Layout(width='10%', height='30px')), start_hour, start_minute])
hbox2 = widgets.HBox([start_stop, end_stop])
hbox3 = widgets.HBox([widgets.Label(value='Confidence interval (%) :'),conf, valid])
ui = widgets.VBox([hbox0, hbox1, hbox2, hbox3])
display(ui, out)

Output()